# AOC 2022

Welcome to the Advent of Code 2022 !

## Basic configuration



In [ ]:
# help for aocd : https://pypi.org/project/advent-of-code-data/

#!pip install aocd

In [ ]:
import os

# replace by your login session cookie
os.environ[
    "AOC_SESSION"
] = ""  # your login session cookie

In [ ]:
from aocd import submit
from aocd.models import Puzzle

In [ ]:
import numpy as np
from tqdm import tqdm
import json
import typing as tp
from collections import Counter, defaultdict, deque
import math
from itertools import product
import re
import string
import matplotlib.pyplot as plt

## Day 16
https://adventofcode.com/2022/day/16
### Prepare input 

## Day 15
https://adventofcode.com/2022/day/15
### Prepare input 

In [ ]:
puzzle = Puzzle(year=2022,day=15)
content = puzzle.input_data.split("\n")

In [ ]:
def l1_distance(x1,y1,x2,y2):
    return abs(x1-x2)+abs(y1-y2)

In [ ]:
class Sensor:
    def __init__(self, x,y,bx,by): 
        self.x = x
        self.y = y
        self.bx = bx
        self.by = by
        self.beacon_distance = l1_distance(self.x,self.y,self.bx,self.by) 
        self.min_x = self.x - self.beacon_distance
        self.max_x = self.x + self.beacon_distance
        
    def check_pos(self, pos):
        return l1_distance(self.x,self.y, pos[0],pos[1]) <= self.beacon_distance
    
    def get_outer_edge(self):
        edge = set()
        for d in range(-self.beacon_distance-1, self.beacon_distance+2):
            diff = self.beacon_distance + 1 - abs(d)
            x = self.x + d
            y1 = self.y + diff
            y2 = self.y - diff 
            if 0 <= x <= 4000000: 
                if 0 <= y1 <= 4000000:
                    edge.add((x,y1))
                if 0 <= y2 <= 4000000:
                    edge.add((x,y2))
        
        return edge
            

In [ ]:
pattern = r"Sensor at x=(-?\d+), y=(-?\d+): closest beacon is at x=(-?\d+), y=(-?\d+)"

sensors = []

for idx, line in enumerate(content): 
    match = re.search(pattern, line)
    sx = match.group(1)
    sy = match.group(2)
    bx = match.group(3)
    by = match.group(4)
    sensors.append(Sensor(int(sx),int(sy),int(bx),int(by)))

### Part 1

In [ ]:
start_x = min(s.min_x for s in sensors)
end_x = max(s.max_x for s in sensors)

In [ ]:
y = 2000000

positions = []
for x in tqdm(range(start_x, end_x+1)): 
    for s in sensors: 
        if s.check_pos((x,y)): 
            positions.append(x)

In [ ]:
n_beacons = len(set([s.bx for s in sensors if s.by == y]))

In [ ]:
answ = len(set(positions))-n_beacons
answ

In [ ]:
puzzle.answer_a = answ

### Part 2

In [ ]:
outer_edges = set().union(*[s.get_outer_edge() for s in sensors])

In [ ]:
for pos in tqdm(outer_edges):
    for s in sensors: 
        if s.check_pos(pos): 
                break 
    else: 
        print(f"Position found {pos}")
        raise
                 

In [ ]:
def frequency(x,y):
    return x * 4000000 + y

In [ ]:
answ = frequency(3267801,2703981)
puzzle.answer_b = answ

## Day 14
https://adventofcode.com/2022/day/14
### Prepare input 

In [ ]:
puzzle = Puzzle(year=2022,day=14)
content = puzzle.input_data.split("\n")

In [ ]:
def draw_line(pos1,pos2): 
    x_start=min(pos1[0],pos2[0])
    x_end=max(pos1[0],pos2[0])
    y_start=min(pos1[1],pos2[1])
    y_end=max(pos1[1],pos2[1])
    
    return [(x,y) for x in range(x_start, x_end+1) for y in range(y_start,y_end+1)]

In [ ]:
def prepare_scan(grid): 
    for cont in content: 
        coords = cont.split(" -> ")
        for c1,c2 in zip(coords, coords[1:]): 
            c1 = tuple(map(int, c1.split(",")))
            c2 = tuple(map(int, c2.split(",")))
            line = draw_line(c1,c2)
            for l in line: 
                grid[l] = "#"
    return grid

In [ ]:
scan = prepare_scan(defaultdict(lambda:"."))

### Part 1

In [ ]:
def step(grid, pos): 
    x,y = pos    
    if grid[(x,y+1)] in {"#", "o"}:
        if grid[(x-1,y+1)] in {"#", "o"}: 
            if grid[(x+1,y+1)] in {"#", "o"}: 
                return pos
            return (x+1,y+1)
        return (x-1,y+1)
    return (x,y+1)


In [ ]:
def move_one_sand(grid, bottom): 
    cur_pos = (500,0)
    new_pos = step(grid,cur_pos)
    while new_pos != cur_pos: 
        cur_pos = new_pos
        new_pos = step(grid, cur_pos)
        if new_pos[1] > bottom:
            return "Stop"
    grid[new_pos] = "o"
    return grid

In [ ]:
bottom = max(k[1] for k in scan.keys())

In [ ]:
count = 0
while True: 
    scan = move_one_sand(scan, bottom)
    if scan == "Stop": 
        break
    count += 1    

In [ ]:
count

In [ ]:
puzzle.answer_a = count

### Part 2

In [ ]:
class DefaultDict(defaultdict):
    def __missing__(self, key):
        return self.default_factory(key)

In [ ]:
FLOOR = bottom+2

In [ ]:
def set_default(key): 
    return "#" if key[1] == FLOOR else "."

scan_2 = prepare_scan(DefaultDict(set_default))

In [ ]:
def move_one_sand_2(grid): 
    cur_pos = (500,0)
    new_pos = step(grid,cur_pos)
    if cur_pos == new_pos: 
        return "Stop"
    
    while new_pos != cur_pos: 
        cur_pos = new_pos
        new_pos = step(grid, cur_pos)
        
    grid[new_pos] = "o"
    return grid

In [ ]:
count = 0
while True: 
    scan_2 = move_one_sand_2(scan_2)
    count += 1 
    if scan_2 == "Stop": 
        break
       

In [ ]:
count

In [ ]:
puzzle.answer_b = count

## Day 13
https://adventofcode.com/2022/day/13
### Prepare input 

In [ ]:
puzzle = Puzzle(year=2022,day=13)
content = [[eval(el) for el in line.split("\n")] for line in puzzle.input_data.split("\n\n")]

### Part 1

In [ ]:
def compare(left,right): 
    if type(left) != type(right): 
        if isinstance(left, int): 
            left = [left]
        else: 
            right = [right]
        
    if isinstance(left,int) and isinstance(right,int):
        return left < right if left != right else None
        
    for l,r in zip(left,right): 
        comp = compare(l,r)
        if comp is None: 
            continue
        return comp
    
    return len(left) < len(right) if len(left) != len(right) else None


In [ ]:
right_pairs = []

for idx, (left, right) in enumerate(content): 
    comp = compare(left, right)
    if comp: 
        right_pairs.append(idx+1)

answ = sum(right_pairs)
answ

In [ ]:
puzzle.answer_a = answ

### Part 2

In [ ]:
content_2 = "\n".join(puzzle.input_data.split("\n\n")).split("\n")

In [ ]:
content_2.extend(["[[2]]", "[[6]]"])

In [ ]:
def compare_2(left, right): 
    comp = compare(eval(left), eval(right))
    if comp is None:
        return 0
    if comp:
        return 1
    return -1

In [ ]:
from functools import cmp_to_key

In [ ]:
content_2.sort(key=cmp_to_key(compare_2), reverse=True)

In [ ]:
answ = (content_2.index("[[2]]") + 1) * (content_2.index("[[6]]") + 1)

In [ ]:
answ

In [ ]:
puzzle.answer_b = answ

## Day 12
https://adventofcode.com/2022/day/12
### Prepare input 

In [ ]:
puzzle = Puzzle(year=2022,day=12)
content = puzzle.input_data.split("\n")

In [ ]:
def get_neighbours(pos:tuple,max_pos:tuple)->list: 
    y,x = pos
    max_y,max_x = max_pos
    top = None if y == 0 else (y-1,x)
    bottom = None if y == max_y else(y+1,x)
    left = None if x==0 else (y,x-1)
    right = None if x==max_x else (y,x+1)
    
    return [top, bottom, left, right]

In [ ]:
height_graph = {}

max_pos = (len(content)-1,len(content[0])-1)

for ldx, line in enumerate(content): 
    for cdx, char in enumerate(line): 
        pos = (ldx,cdx)
        if char == "S": 
            start_pos = pos
            cur_h = 0
        elif char == "E":
            end_pos = pos
            cur_h = 25
        else:
            cur_h = int(string.ascii_lowercase.rfind(char))
            
        neighbours = get_neighbours(pos,max_pos)
        ok_neighbours = []
        for n in neighbours: 
            if n is not None:
                n_h = int(string.ascii_lowercase.rfind(content[n[0]][n[1]]))
                if n_h<=cur_h+1:
                    ok_neighbours.append(n)
        
        height_graph[pos]={"height":cur_h, "neighbours":ok_neighbours}

### Part 1

In [ ]:
def bfs(graph:dict, start:tuple,end:tuple)->list: 
        all_paths = [[start]]
        path_idx = 0

        previous_nodes = {start}
        if start == end:
            return path[0]

        while path_idx < len(all_paths):
            
            cur_path = all_paths[path_idx]
            
            last_node = cur_path[-1]
            next_nodes = graph[last_node]["neighbours"]

            if end in next_nodes:
                cur_path.append(end)
                return cur_path
            
            for next_node in next_nodes:
                if not next_node in previous_nodes:
                    new_path = cur_path[:]
                    new_path.append(next_node)
                    all_paths.append(new_path)
                    
                    previous_nodes.add(next_node)
            
            path_idx += 1
        
        return []

In [ ]:
shortest_path = bfs(height_graph, start_pos, end_pos)

In [ ]:
answ = len(shortest_path)-1
answ

In [ ]:
puzzle.answer_a = answ

### Part 2

In [ ]:
starts = [pos for pos, info in height_graph.items() if info["height"]==0]

In [ ]:
path_lengths = []
for s in tqdm(starts): 
    short = bfs(height_graph, s, end_pos)
    if len(short) != 0:
        path_lengths.append(len(short))
        
answ=min(path_lengths)-1
answ

In [ ]:
puzzle.answer_b = answ

## Day 11
https://adventofcode.com/2022/day/11
### Prepare input 

In [ ]:
puzzle = Puzzle(year=2022,day=11)
puzzle_monkeys = puzzle.input_data.split("\n\n")

In [ ]:
class Monkey: 
    def __init__(self, content: str): 
        content = content.split("\n")
        self.idx = int(content[0][-2])
        self.start_items = deque([int(x) for x in content[1].split(":")[1].strip().split(",")])
        self._operation = content[2].split("=")[1].strip()
        
        self._test_div = int(content[3].replace("Test: divisible by", "").strip())
        self._test_true = int(content[4][-1])
        self._test_false = int(content[5][-1])
        
        self.n_inspected = 0
        
    def operation(self, old:str)->int: 
        return eval(self._operation.replace("old", str(old)))
        
    def test(self, worry_level:int)->int:
        if worry_level % self._test_div == 0: 
            return self._test_true
        return self._test_false

### Part 1

In [ ]:
def do_one_round(monkeys:list, part:tp.Optional[int]=1,common_mult:tp.Optional[int]=None)->list:
    for monkey in monkeys: 
        if len(monkey.start_items) == 0:
            continue
            
        while len(monkey.start_items) != 0: 
            item_worry = monkey.start_items.popleft()
            item_worry = monkey.operation(item_worry)
            monkey.n_inspected += 1
            
            item_worry = item_worry // 3 if part == 1 else item_worry % common_mult
            
            new_monkey_idx = monkey.test(item_worry)
            monkeys[new_monkey_idx].start_items.append(item_worry)
            
    return monkeys

In [ ]:
def monkey_business(monkeys:list)->int: 
    return math.prod(sorted([m.n_inspected for m in monkeys], reverse=True)[:2])

In [ ]:
monkeys = [Monkey(m) for m in puzzle_monkeys]
for _ in range(20): 
    monkeys = do_one_round(monkeys)

In [ ]:
answ = monkey_business(monkeys) 
answ

In [ ]:
puzzle.answer_a = answ

### Part 2

In [ ]:
monkeys = [Monkey(m) for m in puzzle_monkeys]
common_multiple = math.prod(m._test_div for m in monkeys)

for _ in tqdm(range(10000)): 
    monkeys = do_one_round(monkeys, 2, common_multiple)

In [ ]:
answ = monkey_business(monkeys) #25590400731
answ

In [ ]:
puzzle.answer_b = answ

## Day 10
https://adventofcode.com/2022/day/10
### Prepare input 

In [ ]:
puzzle = Puzzle(year=2022, day=10)
content = puzzle.input_data.split("\n")

### Part 1

In [ ]:
def compute_signal(n:int,x:int)->int: 
    if n%40==20: 
        return n*x
    return 0

In [ ]:
x = 1
strenghts = []
n_cycle = 0
for line in content: 
    if line=="noop": 
        n_cycle +=1
        strenghts.append(compute_signal(n_cycle, x))
    else: 
        n_cycle += 1
        strenghts.append(compute_signal(n_cycle, x))
        n_cycle += 1
        strenghts.append(compute_signal(n_cycle, x))           
        _, v = line.split(" ")
        x += int(v)
        

In [ ]:
answ = sum(strenghts)

In [ ]:
answ

In [ ]:
puzzle.answer_a = answ

### Part 2

In [ ]:
def sprite(x:int)->tuple:
    return (x-1,x,x+1)

In [ ]:
def draw_crt(crt_pos:int, x:int)->int: 
    return 1 if crt_pos%40 in sprite(x) else 0

In [ ]:
x = 1
n_cycle = 0
crt = []
for line in content:
    if line == "noop": 
        crt.append(draw_crt(n_cycle, x))
        n_cycle += 1
    else: 
        crt.append(draw_crt(n_cycle, x))
        n_cycle += 1
        
        crt.append(draw_crt(n_cycle, x))
        n_cycle += 1
        _, v = line.split(" ")
        x += int(v)

In [ ]:
crt = np.array(crt).reshape(6, 40)

In [ ]:
plt.imshow(crt)

In [ ]:
puzzle.answer_b = "EFGERURE"

## Day 9
https://adventofcode.com/2022/day/9
### Prepare input 

In [ ]:
puzzle = Puzzle(year=2022, day=9)
content =  puzzle.input_data.split("\n")
new_content = []
for line in content: 
    d, n = line.split(" ")
    new_content.append((d, int(n)))

In [ ]:
new_content

### Part 1

In [ ]:
def add_tuple(tup1:tuple, tup2:tuple)->tuple: 
    return tuple(map(lambda i, j: i + j, tup1, tup2))

In [ ]:
def move(pos:tuple, movement:tuple)->tuple: 
    return add_tuple(pos, movement)

In [ ]:
def move_closer(pos1:tuple, pos2:tuple)->tuple:
    diff_x = pos2[0] - pos1[0]
    diff_y = pos2[1] - pos1[1]
    
    if abs(diff_x)>1:
        move_x = math.copysign(1,diff_x) 
        move_y = math.copysign(1,diff_y) if abs(diff_y)>=1 else 0
        
    elif abs(diff_y)>1:
        move_x = math.copysign(1,diff_x) if abs(diff_x)>=1 else 0
        move_y = math.copysign(1,diff_y) 
    else:
        move_x = 0
        move_y = 0
    
    return add_tuple(pos1, (move_x, move_y))

In [ ]:
def move_head(pos_h:tuple, direction:str)->tuple: 
    if direction == "U": 
        pos_h = move(pos_h, (0,1))
    elif direction == "D": 
        pos_h = move(pos_h, (0,-1))
    elif direction == "L":
        pos_h = move(pos_h, (-1,0))
    else: 
        pos_h = move(pos_h, (1,0))
    return pos_h

In [ ]:
pos_h = (0,0)
pos_t = (0,0)

t_positions = [pos_t]
for direction, n in new_content:
    for _ in range(n):
        pos_h = move_head(pos_h, direction)
        
        pos_t = move_closer(pos_t, pos_h)
        t_positions.append(pos_t)


In [ ]:
answ = len(set(t_positions))
answ

In [ ]:
puzzle.answer_a = answ

### Part 2

In [ ]:
cur_positions = [(0,0)]*10
t_positions = [(0,0)]

for direction, n in new_content: 
    for _ in range(n): 
        new_positions = [move_head(cur_positions[0], direction)]
        for i in range(len(cur_positions)-1):
            new_pos = move_closer(cur_positions[i+1], new_positions[i])
            new_positions.append(new_pos)
        t_positions.append(new_positions[-1])
        cur_positions = new_positions

In [ ]:
answ = len(set(t_positions))
answ

In [ ]:
puzzle.answer_b = answ

## Day 8
https://adventofcode.com/2022/day/8
### Prepare input 

In [ ]:
puzzle = Puzzle(year=2022, day=8)
content = puzzle.input_data.split("\n")

In [ ]:
content =[[int(i) for i in line] for line in content]

In [ ]:
class Tree: 
    def __init__(self, x:int, y:int, array:np.array): 
        self.x = x
        self.y = y
        self.array = array
        self.h = array[y,x]
        self.on_edge = True if (x in {0, len(array[0])-1} or y in {0, len(array)-1}) else False
        self._is_seen = None
        self._scenic_score = None
        
    def check_left(self)->bool: 
        return all(h < self.h for h in self.array[self.y,:self.x])
    def check_right(self)->bool: 
        return all(h < self.h for h in self.array[self.y,self.x + 1:])
    def check_above(self)->bool: 
        return all(h < self.h for h in self.array[:self.y,self.x])
    def check_below(self)->bool: 
        return all(h < self.h for h in self.array[self.y + 1:,self.x])
    
    def is_seen(self)->bool:
        if self._is_seen is None:
            self._is_seen = True if self.on_edge else any([self.check_left(), self.check_right(), self.check_above(), self.check_below()])
        return self._is_seen

    
    def count(self, step_x:int, step_y:int)->int: 
        cur_x = self.x+step_x
        cur_y = self.y+step_y
        count = 0
        while cur_x >= 0 and cur_x < len(self.array[0]) and cur_y >= 0 and cur_y < len(self.array):
            cur_h = self.array[cur_y, cur_x]
            count += 1
            if cur_h >= self.h: 
                break
            cur_x += step_x
            cur_y += step_y
        return count
    
    def look_left(self)->int:
        if self.x == 0:
            return 0
        return self.count(-1, 0)
    def look_right(self)->int:
        if self.x == len(array[0])-1:
            return 0
        return self.count(1,0)
    def look_up(self)->int: 
        if self.y == 0:
            return 0
        return self.count(0, -1)
    def look_down(self)->int: 
        if self.y == len(array)-1:
            return 0
        return self.count(0,1)
    
    def scenic_score(self)->int:
        if self._scenic_score is None: 
            self._scenic_score = math.prod([self.look_left(), self.look_right(), self.look_up(), self.look_down()])
        return self._scenic_score

In [ ]:
array = np.array(content)

In [ ]:
tree_grid = [Tree(x,y,array) for y in range(len(array)) for x in range(len(array[0]))]
    

### Part 1

In [ ]:
answ = sum(t.is_seen() for t in tree_grid)

In [ ]:
answ

In [ ]:
puzzle.answer_a = answ

### Part 2

In [ ]:
answ = max(t.scenic_score() for t in tree_grid)
answ

In [ ]:
puzzle.answer_b = answ

## Day 7
https://adventofcode.com/2022/day/7
### Prepare input 

In [ ]:
puzzle = Puzzle(year=2022, day=7)
content = puzzle.input_data.split("\n")

In [ ]:
class Directory: 
    def __init__(self, path:str, parent_dir:tp.Optional[str]=None): 
        self.path = path
        self.parent_dir = parent_dir
        self.subdirs = []
        self.file_weight = 0
        self._total_weight = 0
    
    def total_weight(self): 
        if self._total_weight == 0: 
            
            subweight = sum(subd.total_weight() for subd in self.subdirs) if len(self.subdirs) != 0 else 0
            self._total_weight = self.file_weight + subweight
            
        return self._total_weight
    

In [ ]:
def get_path(directory: Directory, name:str)->str: 
    return os.path.join(directory.path, name) if directory is not None else name

In [ ]:
created = {}

idx = 0
cur_dir = None

for line in content:
    cur_line = line.split(" ")
    if cur_line[0]=="$" and cur_line[1]=="ls": 
        continue
    elif cur_line[0]=="$" and cur_line[1]=="cd":
        name = cur_line[2]
        path = get_path(cur_dir, name)
        if name == "..": 
            cur_dir = cur_dir.parent_dir 
        elif path in created.keys(): 
            cur_dir = created[path]
        else: 
            cur_dir = Directory(path, cur_dir)
            created[path] = cur_dir
    elif cur_line[0] == "dir": 
        name = cur_line[1]
        path = get_path(cur_dir, name)
         
        new_dir = Directory(path, cur_dir)
        created[path] = new_dir
        cur_dir.subdirs.append(new_dir)
    else: 
        cur_dir.file_weight += int(cur_line[0])

### Part 1

In [ ]:
answ = sum(curdir.total_weight() for curdir in created.values() if curdir.total_weight()<=100000)
answ

In [ ]:
puzzle.answer_a = answ

### Part 2

In [ ]:
free = 70000000 - created["/"].total_weight()
needed = 30000000 - free
needed

In [ ]:
answ = min(curdir.total_weight() for curdir in created.values() if curdir.total_weight()>=needed)
answ

In [ ]:
puzzle.answer_b = answ

## Day 6
https://adventofcode.com/2022/day/6
### Prepare input

In [ ]:
puzzle = Puzzle(year=2022, day=6)
content = puzzle.input_data

### Part 1

In [ ]:
def find_marker(content:str, start_idx:int)->int: 
    marker = deque(content[:start_idx-1])
    idx = start_idx-1
    while True: 
        marker.append(content[idx])
        if len(set(marker)) == len(marker): 
            break
        marker.popleft()
        idx += 1

    return idx + 1


In [ ]:
puzzle.answer_a = find_marker(content, 4)

### Part 2

In [ ]:
puzzle.answer_b = find_marker(content, 14)

## Day 5
https://adventofcode.com/2022/day/5
### Prepare input

In [ ]:
puzzle = Puzzle(year=2022, day=5)

In [ ]:
stacks,procedure = puzzle.input_data.split("\n\n")

In [ ]:
def prepare_stacks(stacks:list)-> list:
    stacks = stacks.split("\n")[:-1]
    transpose_stacks = [''.join(s) for s in zip(*stacks)][1::4]
        
    new_stacks = [deque(s.strip()) for s in transpose_stacks]
    for d in new_stacks:
        d.reverse()
        
    return new_stacks

In [ ]:
new_stacks = prepare_stacks(stacks)

In [ ]:
procedure = procedure.split("\n")
new_procedure = [line.strip("move ").replace(" from ",",").replace(" to ", ",") for line in procedure]
new_procedure = [[int(x) for x in line.split(",")] for line in new_procedure]

### Part 1

In [ ]:
def apply_procedure(stacks:list, procedure:list, part:tp.Optional[int]=1)->str: 
    for n_move, col1, col2 in procedure: 
        if part == 1:
            for _ in range(n_move): 
                stacks[col2-1].append(stacks[col1-1].pop())
        else:
            substack = []
            for _ in range(n_move): 
                substack.append(stacks[col1-1].pop())
            substack.reverse()
            stacks[col2-1].extend(substack)
        
    return "".join(d[-1] for d in stacks)

In [ ]:
answ = apply_procedure(new_stacks, new_procedure)

In [ ]:
puzzle.answer_a = answ

### Part 2

In [ ]:
new_stacks = prepare_stacks(stacks)
answ = apply_procedure(new_stacks, new_procedure, part=2)

In [ ]:
puzzle.answer_b = answ

## Day 4
https://adventofcode.com/2022/day/4
### Prepare input 

In [ ]:
puzzle = Puzzle(year=2022, day=4)
content = puzzle.input_data.split("\n")

In [ ]:
content

In [ ]:
def get_int_range(str_range:str)-> list: 
    start, stop = str_range.split("-")
    return set(i for i in range(int(start), int(stop)+1))

In [ ]:
new_content = [[get_int_range(el) for el in pair.split(",")] for pair in content]

### Part 1 

In [ ]:
count = sum(r1.issubset(r2) or r2.issubset(r1) for r1,r2 in new_content)
count

In [ ]:
puzzle.answer_a = count

### Part 2

In [ ]:
count = sum(bool(r1&r2) for r1,r2 in new_content)
count

In [ ]:
puzzle.answer_b = count

## Day 3
https://adventofcode.com/2022/day/3
### Prepare input

In [ ]:
puzzle = Puzzle(year=2022, day=3)
content = puzzle.input_data.split("\n")

In [ ]:
content

### Part 1

In [ ]:
def find_item(items:str)->str: 
    half = int(len(items)/2)
    comp1 = items[:half]
    comp2 = items[half:]
    
    return list(set(comp1)&set(comp2))[0]

In [ ]:
ALPHABET = string.ascii_lowercase+string.ascii_uppercase

def get_priority(item:str)->int:
    return ALPHABET.rfind(item)+1

In [ ]:
answ = sum(get_priority(find_item(items)) for items in content)
answ

In [ ]:
puzzle.answer_a = answ

### Part 2

In [ ]:
def find_group_item(group:list)->str: 
    it1, it2, it3 = group
    return list(set(it1)&set(it2)&set(it3))[0]

In [ ]:
answ = sum(get_priority(find_group_item(content[i:i+3])) for i in range(0,len(content),3))
answ

In [ ]:
puzzle.answer_b = answ

## Day 2
https://adventofcode.com/2022/day/2
### Prepare input

In [ ]:
puzzle = Puzzle(year=2022, day=2)
content = [line.split(" ") for line in puzzle.input_data.split("\n")]
content

### Part 1

In [ ]:
def replace_play(play:str) -> int: 
    if play in ["A", "X"]: 
        return 1
    if play in ["B", "Y"]: 
        return 2
    if play in ["C", "Z"]:
        return 3
    
def replace_pair(pair:list) -> list: 
    return [replace_play(x) for x in pair]

In [ ]:
def compute_score(pair:list) -> int: 
    p1,p2 = replace_pair(pair)
    if (p1-p2)%3 == 1:
        return p2
    if p1 == p2 : 
        return p2 + 3
    return p2 + 6  

In [ ]:
answ = sum(compute_score(pair) for pair in content)
answ

In [ ]:
puzzle.answer_a = answ

### Part 2

In [ ]:
def compute_score_2(pair:list) -> int: 
    p1, result = pair
    p1 = replace_play(p1)
    
    if result == "X" : 
        p2 = p1-1
        score = p2 if p2 != 0 else 3
    elif result == "Y": 
        score = p1 + 3
    else: 
        p2 = p1+1
        score = p2 if p2!=4 else 1
        score += 6
    return score

In [ ]:
answ = sum(compute_score_2(pair) for pair in content)
answ

In [ ]:
puzzle.answer_b = answ

## Day 1
https://adventofcode.com/2022/day/1

### Prepare input

In [ ]:
puzzle = Puzzle(year=2022, day=1)

In [ ]:
content =[[int(x) for x in elf.split("\n")] for elf in puzzle.input_data.split("\n\n")]
content

### Part 1

In [ ]:
answ = max(sum(elf)for elf in content)
answ

In [ ]:
puzzle.answer_a = answ

### Part 2

In [ ]:
answ = sum(sorted(sum(elf) for elf in content)[-3:])
answ

In [ ]:
puzzle.answer_b = answ